In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import class from database
from MongoCRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "password1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({"animal_type": "Dog"}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id','datetime','monthyear','age'],inplace=True)

## Debug
print(len(df.to_dict(orient='records')))
print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare Logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(className='row',
            style={'display': 'flex'},
            children=[
                html.Div(id='hidden-div', style={'display': 'none'}),
                html.A([
                    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                            height=100, width=100
                            )
                ],
                    href='https://www.snhu.edu', target='_blank'),
                # Unique Identifier
                html.Center(html.B(html.H1(children=[
                'Patrick Cochran',
                html.Br(),
                'SNHU - CS-340',
                html.Br(),
                'Grazioso Salvare Dashboard'
                ])))
            ]
            ),
    html.Hr(),
    
    html.Div([
        html.H1('Rescue Dog Type'),
        #Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.RadioItems(
        # Id for callbacks
        id='data_filter',
        options=[
        {'label': 'Water', 'value': 'Water'},
        {'label': 'Mountain or Wilderness', 'value': 'Mountain or Wilderness'},
        {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
        {'label': 'Reset', 'value': 'None'},
        ],
        value = 'None' ,
        inline=True
        )
    ], style={'display': 'horizontal', 'alignItems':'left', 'justifyContent':'left'}),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#Features for interactive data table
        # Single row selection
        row_selectable='single',
        # Enable sorting
        sort_action='native',
        # Enable filtering
        filter_action='native',
        # Enable page parameters
        page_action='native',
        # Current page number
        page_current=0,
        # Shown rows per page
        page_size=15,
        
        # Horizontal scroll
        style_table={'overflowX': 'auto'},
        # Enhansed text visibility
        style_cell={
            'minWidth': '180px', 'width': '140px', 'maxWidth': '180px',
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
        },
        # Differentiation of header style
        style_header={
            'backgroundColor': 'white',
            'fontWeight': 'bold'
        },
    ),

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            dcc.Graph(id='bar-chart'),
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    #unique signature with Github link
    html.Div([
            html.P("Patrick Cochran", style={'margin-left':'10px', 'font-size':'16px'}),
            html.P([
                'Github Link: ',
                html.A("kinstinct", href='https://github.com/kinstinct', target="_blank")
            ], style={'margin-left': '10px', 'font-size': '16px'})
    ], style = {'text-align': 'left', 'margin-top':'10px'})
    
])
           

#############################################
# Interaction Between Components / Controller
#############################################



# Highlight row when selected    
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
    )
## Code to filter interactive data table with MongoDB queries
def update_styles(selected_columns):
    if selected_columns is None:
        #If none, return empty list
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Reset selections on table
@app.callback(
    # Update selected rows
    Output('datatable-id', 'selected_rows'),
    # Value changes given over radio
    [Input('data_filter', 'value')]
)
def reset_table_selection(selected_type):
    #Return blank list
    return []

# Update pie chart & update breeds
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('datatable-id', 'data')]
)
def update_breed_distribution_chart(data):
    if data is None or not data:
        return px.bar(title="No Data Available")
    
    df = pd.DataFrame(data)
    if 'breed' in df.columns:
        # Aggregate data to count the frequency of each breed
        breed_counts = df['breed'].value_counts().reset_index()
        breed_counts.columns = ['breed', 'count']
        
        # Create horizontal bar chart
        fig = px.bar(breed_counts, x='count', y='breed', orientation='h',
                     title="Distribution of Dog Breeds",
                     labels={'count':'Number of Dogs', 'breed': 'Dog Breed'})
        fig.update_layout(
            autosize=False,
            width=1000,
            height=500,
            margin=dict(l=0, r=0, t=30, b=0),
            xaxis_title="Number of Dogs",
            yaxis_title=None
        )
        return fig
    else:
        return px.bar(title="No Data Available for Breeds")

# Update data in the radio menu based on animal type slection    
@app.callback(
    # Update datatable
    Output('datatable-id', 'data'),
    # Changes played on radio
    [Input('data_filter', 'value')]
)
def update_table(selected_type):
    if selected_type == 'None':
        query = {"animal_type": "Dog"}
        
    elif selected_type == 'Water':
        query = {
            "animal_type": "Dog",
        
            "breed": {"$regex": "Labrador Retriever Mix|Chesapeake Bay Retriever|NewfoundLand","$options": "i"},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
        
    elif selected_type == 'Mountain or Wilderness':
        query = {
            "animal_type": "Dog",
            
            'breed': {"$regex": "German Shepherd|Alaska Malamute|Siberian Husky|Rottweiler","$options": "i"},
            "sex_upon_outcome": "Intact Male" ,
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
        
    elif selected_type == 'Distater or Individual Tracking':
        query = {
            "animal_type": "Dog",
            
            "breed": {"$regex": "Doberman Pinscher|German Shepherd|Golden Retriever|Bloodhound|Rottweiler","$options": "i"},
            "sex_upon_outcome": "Intact Male" ,
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:
        query = {"animal_type": "Dog"}
            
    # Get new data from MongoDB
    new_df = pd.DataFrame.from_records(db.read(query))
    # Remove _id column
    new_df.drop(columns=['_id','datetime','monthyear'], inplace=True)
    # Update table data
    return new_df.to_dict('records')


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, selected_rows):  
    # Set default for center and zoom
    center = [20,0]
    zoom = 2
    
    if viewData is None or not selected_rows:
        # If on row selected, return default
        return [dl.Map(center=center, zoom=zoom, children=[dl.TileLayer()], style={'width':'1000px', 'height':'500px'})]

    # Access selected rows
    rows = pd.DataFrame.from_records(viewData)
                       
    if selected_rows and viewData and selected_rows[0] < len(rows):
              
        selected_row = rows.iloc[selected_rows[0]]
        latitude = selected_row['location_lat']
        longitude = selected_row['location_long']
        center = [latitude, longitude]
        zoom = 10
                       
        # Create marker
        marker = dl.Marker(position=center, children=[
            dl.Tooltip(selected_row['breed']),
            dl.Popup(html.Div([
                html.B("Animal Name:"),
                html.Br(),
                html.Div(selected_row['name'])
            ]))
        ])
                       
        # Update to include new marker
        return [dl.Map(center=center, zoom=zoom, children=[
            dl.TileLayer(),
            marker
        ], style={'width': '1000px', 'height': '500px'})]

app.run_server(debug=True)

11178
Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'name', 'outcome_subtype', 'outcome_type',
       'sex_upon_outcome', 'location_lat', 'location_long',
       'age_upon_outcome_in_weeks'],
      dtype='object')
Dash app running on http://127.0.0.1:15046/
